<a href="https://colab.research.google.com/github/VaishKS/MNIST_CNN_PyTorch/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>